In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_filepath = r"C:\Users\emanlangit\Documents\2025\00 ACSIN\Model Refresh\XGB_only\Two_Wheelers_only\Raw_Data\EDA_df.csv"
df_dev = pd.read_csv(data_filepath, delimiter = ",")

data_filepath_testcases = r"C:\Users\emanlangit\Documents\2025\00 ACSIN\Model Refresh\XGB_only\Two_Wheelers_only\Raw_Data\SCM_Data_21-02-25_Rev_1.28_input.csv"
df_testcases = pd.read_csv(data_filepath_testcases, delimiter = ",")

In [3]:
df_dev = df_dev[df_dev.product_description == 'Two-Wheeler']
df_dev = df_dev[df_dev['total_income'] >= 1000]
df_dev = df_dev[df_dev['loan_account_no'] != 'TW01010103000002258833']

df_dev.columns = df_dev.columns.str.lower()

#standardize values capitalization
df_dev = df_dev.map(lambda x: x.upper() if isinstance(x, str) else x)

#remove trailing and leading spaces
obj_cols = list(df_dev.select_dtypes('object'))
df_dev[obj_cols] = df_dev[obj_cols].apply(lambda category: category.str.strip()) 

#remove spaces in between values, replace with underscore
df_dev = df_dev.map(lambda x: '_'.join(x.split()) if isinstance(x, str) else x)
df_dev.shape

(49681, 35)

In [4]:
df_dev.columns

Index(['application_number', 'customer_no', 'product_description',
       'loan_account_no', 'requested_loan_amount', 'original_emi_amount',
       'res_years_at_current_city', 'res_resident_status', 'res_city',
       'res_state', 'age', 'date_of_birth', 'manufacturer',
       'down_payment_amount', 'total_income', 'branch_name', 'occupation_type',
       'industry', 'interest_rate', 'number_of_installments',
       'loan_asset_cost', 'bureau_score', 'asset_category', 'asset_model',
       'dealer', 'branch_id', 'channel', 'final_sanctioned_amount',
       'disbursal_amount', 'res_years_at_current_address', 'gender',
       'marital_status', 'nature_of_business', 'years_in_occupation',
       'loan_application_date'],
      dtype='object')

In [5]:
df_testcases.shape

(18623, 40)

In [6]:
#test data reduce to valid LANs only
temp_df_dev = df_dev.copy() #raw dev data
temp_df_dev = temp_df_dev.rename(columns={'loan_account_no':'loan_account_no_og'}) #rename identifier
new_col12m = df_testcases.copy() #raw test cases
new_col12m = new_col12m.rename(columns={'LOAN_ACCOUNT_NO_og':'loan_account_no_og'}) #rename identifier
new_col12m = new_col12m.dropna(subset =['loan_account_no_og']) #no missing LAN but with duplicate
duplicated_A = new_col12m.duplicated(subset='loan_account_no_og', keep =False)
tagged_LANs_result_no_dup = new_col12m.drop(new_col12m[duplicated_A].index)
tagged_LANs_result_no_dup_TW = tagged_LANs_result_no_dup[tagged_LANs_result_no_dup.product_description == '2 Wheeler Loan']
tagged_LANs_result_no_dup_TW.shape

(6928, 40)

In [7]:
final_test_data = tagged_LANs_result_no_dup_TW.copy()

final_test_data.columns = final_test_data.columns.str.lower()

#standardize values capitalization
final_test_data = final_test_data.map(lambda x: x.upper() if isinstance(x, str) else x)

#remove trailing and leading spaces
obj_cols = list(final_test_data.select_dtypes('object'))
final_test_data[obj_cols] = final_test_data[obj_cols].apply(lambda category: category.str.strip()) 

#remove spaces in between values, replace with underscore
final_test_data = final_test_data.map(lambda x: '_'.join(x.split()) if isinstance(x, str) else x)
#final_test_data.info()

In [8]:
final_test_data.shape

(6928, 40)

In [9]:
final_test_data.columns

Index(['loan_account_no', 'application_number', 'cifnumber', 'customer_number',
       'application received date', 'product_description', 'judgement date',
       'decision', 'user_comment', 'loan_account_no_og',
       'loan_sanctioned_date', 'loan_amount', 'disbursal_date',
       'loan_asset_cost', 'number_of_installments', 'channel',
       'original_emi_amount', 'interest_rate', 'res_years_at_current_city',
       'years_in_occupation', 'res_years_at_current_address',
       'res_resident_status', 'res_city', 'res_state', 'age', 'date_of_birth',
       'gender', 'manufacturer', 'down_payment_amount', 'asset_model',
       'dealer', 'total_income', 'branch_name', 'occupation_type', 'industry',
       'marital_status', 'nature_of_business', 'loan amount requested',
       'bureau_score', 'd3_within12months'],
      dtype='object')

In [10]:
df_dev_appdate = df_dev['loan_application_date'].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors = 'coerce'))
print(f'Earliest date: {df_dev_appdate.min()}')
print(f'Ladev date: {df_dev_appdate.max()}')

Earliest date: 2023-01-01 00:00:00
Ladev date: 2023-12-31 00:00:00


In [11]:
df_test_appdate = final_test_data['application received date'].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y', errors = 'coerce'))
print(f'Earliest date: {df_test_appdate.min()}')
print(f'Latest date: {df_test_appdate.max()}')

Earliest date: 2024-01-01 00:00:00
Latest date: 2024-02-28 00:00:00


In [12]:
#df_dev = df_dev.dropna(subset = 'disbursal_amount')
#df_dev = df_dev[df_dev.disbursal_amount != 0]

df_test = final_test_data.copy()

In [35]:
df_dev['kind'] = "dev"
df_test['kind'] = "test"

df_full = pd.concat([df_dev,df_test])
df_full['dp_ratio'] = df_full['down_payment_amount'] /df_full['loan_asset_cost']
df_full['monthly_income'] = df_full['total_income'] /12

#grouping asset_model - Jan ok
text_list = ['SPLENDOR', 'ACCESS', 'BURGMAN', 'ACTIVA', 'NTORQ', 'JUPITER', 'HF', 'AVENIS']
# Function to check and assign text or 'Others'
def check_text(value):
    value = value.replace('_', ' ')  # Replace underscores with spaces
    for text in text_list:
        if text in value:
            return text
    return 'OTHERS'

# Create new column based on the function
df_full['asset_model_grp'] = df_full['asset_model'].apply(check_text)

In [36]:
df = df_full.copy()

In [37]:
#df_segments
df_b300 = df[df.kind == 'dev']
df_300_749 = df[df.kind == 'test']

In [38]:
print(f' No. of rows below 300: {len(df_b300)}')
print(f' No. of rows 300-749: {len(df_300_749)}')

 No. of rows below 300: 49681
 No. of rows 300-749: 6928


In [39]:
#dfs to compare PSI
working_df1 = df_b300
working_df2 = df_300_749

In [32]:
#bureau_score
def bureau_score_bin(x):
    try:
        y = float(x)
        
        if y >= 750:
            return 'g [750, Inf)'
        elif y >= 731 and y <= 749:
            return 'e [731,749]'
        elif y >= 700 and y <= 730:
            return 'd [700,730]'
        elif y >= 680 and y <= 699:
            return 'c [680,699]'
        elif y >= 300 and y <= 679:
            return 'b [300,679]'
        elif y < 300:
            return 'a NTC'        
    except:
        return x

working_df1.loc[:, 'bureau_score_bin'] = working_df1['bureau_score'].apply(bureau_score_bin)
t1_bureau_score = working_df1['bureau_score_bin'].value_counts().sort_index()

working_df2.loc[:, 'bureau_score_bin'] = working_df2['bureau_score'].apply(bureau_score_bin)
t2_bureau_score = working_df2['bureau_score_bin'].value_counts().sort_index()

bureau_score_df = pd.DataFrame({f'Below 300':t1_bureau_score, 
                           f'300-749':t2_bureau_score})

In [19]:
#dp ratio
def dp_ratio_bin(x):
    try:
        y = float(x)
        
        if y >= 0.25:
            return 'f [0.25, Inf)'
        elif y >= 0.2 and y < 0.25:
            return 'e [0.2,0.25)'
        elif y >= 0.15 and y < 0.2:
            return 'd [0.15,0.2)'
        elif y >= 0.1 and y < 0.15:
            return 'c [0.1,0.15)'
        elif y >= 0.05 and y < 0.1:
            return 'b [0.05,0.1)'
        elif y < 0.05:
            return 'a [-Inf,0.05)'        
    except:
        return x

working_df1['dp_ratio_bin'] = working_df1['dp_ratio'].apply(dp_ratio_bin)
t1_dpratio = working_df1['dp_ratio_bin'].value_counts().sort_index()

working_df2['dp_ratio_bin'] = working_df2['dp_ratio'].apply(dp_ratio_bin)
t2_dpratio = working_df2['dp_ratio_bin'].value_counts().sort_index()

dpratio_df = pd.DataFrame({f'Below 300':t1_dpratio, 
                           f'300-749':t2_dpratio})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3666455650.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['dp_ratio_bin'] = working_df1['dp_ratio'].apply(dp_ratio_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3666455650.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['dp_ratio_bin'] = working_df2['dp_ratio'].apply(dp_ratio_bin)


In [20]:
#original_emi_amount
def orig_emi_amount_bin(x):
    try:
        y = float(x)
        
        if y >= 5500:
            return 'g [5500, Inf)'
        elif y >= 5000 and y < 5500:
            return 'f [5000,5500)'
        elif y >= 4500 and y < 5000:
            return 'e [4500,5000)'
        elif y >= 4000 and y < 4500:
            return 'd [4000,4500)'
        elif y >= 3500 and y < 4000:
            return 'c [3500,4000)'
        elif y >= 3_000 and y < 3_500:
            return 'b [3000,3500)'
        elif y < 3_000:
            return 'a [-Inf,3000)'        
    except:
        return x

working_df1['orig_emi_amt_bin'] = working_df1['original_emi_amount'].apply(orig_emi_amount_bin)
t1_orig_emi_amt = working_df1['orig_emi_amt_bin'].value_counts().sort_index()

working_df2['orig_emi_amt_bin'] = working_df2['original_emi_amount'].apply(orig_emi_amount_bin)
t2_orig_emi_amt = working_df2['orig_emi_amt_bin'].value_counts().sort_index()

orig_emi_amt_df = pd.DataFrame({f'Below 300':t1_orig_emi_amt, 
                                f'300-749':t2_orig_emi_amt})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3271020287.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['orig_emi_amt_bin'] = working_df1['original_emi_amount'].apply(orig_emi_amount_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3271020287.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['orig_emi_amt_bin'] = working_df2['original_emi_amount'].apply(orig_emi_amount_bin)


In [21]:
#down_payment_amount
def down_pay_amt_bin(x):
    try:
        y = float(x)
        
        if y >= 20_000:
            return 'f [20000, Inf)'
        elif y >= 15000 and y < 20000:
            return 'e [15000,20000)'
        elif y >= 12000 and y < 15000:
            return 'd [12000,15000)'
        elif y >= 10_000 and y < 12_000:
            return 'c [10000,12000)'
        elif y >= 8_000 and y < 10_000:
            return 'b [8000,10_000)'
        elif y < 8_000:
            return 'a [-Inf,8000)'        
    except:
        return x

working_df1['down_payment_amount_bin'] = working_df1['down_payment_amount'].apply(down_pay_amt_bin)
t1_down_pay_amt = working_df1['down_payment_amount_bin'].value_counts().sort_index()

working_df2['down_payment_amount_bin'] = working_df2['down_payment_amount'].apply(down_pay_amt_bin)
t2_down_pay_amt = working_df2['down_payment_amount_bin'].value_counts().sort_index()


down_pay_amt_df = pd.DataFrame({f'Below 300':t1_down_pay_amt, 
                                f'300-749':t2_down_pay_amt})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2251460788.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['down_payment_amount_bin'] = working_df1['down_payment_amount'].apply(down_pay_amt_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2251460788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['down_payment_amount_bin'] = working_df2['down_payment_amount'].apply(down_pay_amt_bin)


In [22]:
#loan_asset_cost
def loan_asset_cost_bin(x):
    try:
        y = float(x)
        
        if y >= 115_000:
            return 'g [115K, Inf)'
        elif y >= 110_000 and y < 115_000:
            return 'f [110K,115K)'
        elif y >= 105_000 and y < 110_000:
            return 'e [105K,110K)'
        elif y >= 100_000 and y < 105_000:
            return 'd [100K,105K)'
        elif y >= 95_000 and y < 100_000:
            return 'c [95K,100K)'
        elif y >= 90_000 and y < 95_000:
            return 'b [90K,95K)'
        elif y < 90_000:
            return 'a [-Inf,90K)'        
    except:
        return x

working_df1['loan_asset_cost_bin'] = working_df1['loan_asset_cost'].apply(loan_asset_cost_bin)
t1_loan_asset_cost = working_df1['loan_asset_cost_bin'].value_counts().sort_index()

working_df2['loan_asset_cost_bin'] = working_df2['loan_asset_cost'].apply(loan_asset_cost_bin)
t2_loan_asset_cost = working_df2['loan_asset_cost_bin'].value_counts().sort_index()

loan_asset_cost_df = pd.DataFrame({f'Below 300':t1_loan_asset_cost, 
                                f'300-749':t2_loan_asset_cost})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2967267195.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['loan_asset_cost_bin'] = working_df1['loan_asset_cost'].apply(loan_asset_cost_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2967267195.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['loan_asset_cost_bin'] = working_df2['loan_asset_cost'].apply(loan_asset_cost_bin)


In [23]:
#age
def age_bin(x):
    try:
        y = float(x)
        
        if y >= 50:
            return 'h [50, Inf)'
        elif y >= 45 and y < 50:
            return 'g [45,50)'
        elif y >= 40 and y < 45:
            return 'f [40,45)'
        elif y >= 35 and y < 40:
            return 'e [35,40)'
        elif y >= 30 and y < 35:
            return 'd [30,35)'
        elif y >= 25 and y < 30:
            return 'c [25,30)'
        elif y >= 20 and y < 25:
            return 'b [20,25)'
        elif y < 20:
            return 'a [-Inf,20)'        
    except:
        return x

working_df1['age_bin'] = working_df1['age'].apply(age_bin)
t1_age = working_df1['age_bin'].value_counts().sort_index()

working_df2['age_bin'] = working_df2['age'].apply(age_bin)
t2_age = working_df2['age_bin'].value_counts().sort_index()


age_df = pd.DataFrame({f'Below 300':t1_age, 
                       f'300-749':t2_age})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2397726341.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['age_bin'] = working_df1['age'].apply(age_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2397726341.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['age_bin'] = working_df2['age'].apply(age_bin)


In [24]:
#years_in_occupation
def years_in_occup_bin(x):
    try:
        y = float(x)
        
        if y > 999_999:
            return 'h missing'
        elif y >= 6 and y < 1_000_000:
            return 'c [6,Inf)'
        elif y >= 5 and y < 6:
            return 'b [5,6)'
        elif y < 5:
            return 'a [-Inf,5)'        
    except:
        return x

working_df1['years_in_occupation_bin'] = working_df1['years_in_occupation'].apply(years_in_occup_bin)
t1_years_in_occup = working_df1['years_in_occupation_bin'].value_counts().sort_index()

working_df2['years_in_occupation_bin'] = working_df2['years_in_occupation'].apply(years_in_occup_bin)
t2_years_in_occup = working_df2['years_in_occupation_bin'].value_counts().sort_index()


years_in_occup_df = pd.DataFrame({f'Below 300':t1_years_in_occup, 
                       f'300-749':t2_years_in_occup})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\1476712543.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['years_in_occupation_bin'] = working_df1['years_in_occupation'].apply(years_in_occup_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\1476712543.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['years_in_occupation_bin'] = working_df2['years_in_occupation'].apply(years_in_occup_bin)


In [25]:
#total_income
def total_income_bin(x):
    try:
        y = float(x)
        
        if y >= 550000:
            return 'f [550000, Inf)'
        elif y >= 500000 and y < 550000:
            return 'e [500000,550000)'
        elif y >= 450000 and y < 500000:
            return 'd [450000,500000)'
        elif y >= 400000 and y < 450000:
            return 'c [400000,450000)'
        elif y >= 350000 and y < 400000:
            return 'b [350000,400000)'
        elif y < 350000:
            return 'a [-Inf,350000)'        
    except:
        return x

working_df1['total_income_bin'] = working_df1['total_income'].apply(total_income_bin)
t1_total_income = working_df1['total_income_bin'].value_counts().sort_index()

working_df2['total_income_bin'] = working_df2['total_income'].apply(total_income_bin)
t2_total_income = working_df2['total_income_bin'].value_counts().sort_index()


total_income_df = pd.DataFrame({f'Below 300':t1_total_income, 
                       f'300-749':t2_total_income})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2077899225.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['total_income_bin'] = working_df1['total_income'].apply(total_income_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\2077899225.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['total_income_bin'] = working_df2['total_income'].apply(total_income_bin)


In [26]:
#monthly_income
def monthly_income_bin(x):
    try:
        y = float(x)
        
        if y >= 1_000_000:
            return 'e missing'
        elif y >= 40_000 and y < 1_000_000:
            return 'd [40K, Inf)'
        elif y >= 35_000 and y < 40_000:
            return 'c [35K,40K)'
        elif y >= 30_000 and y < 35_000:
            return 'b [30K,35K)'
        elif y < 30_000:
            return 'a [-Inf,30K)'        
    except:
        return x

working_df1['monthly_income_bin'] = working_df1['monthly_income'].apply(monthly_income_bin)
t1_monthly_income = working_df1['monthly_income_bin'].value_counts().sort_index()

working_df2['monthly_income_bin'] = working_df2['monthly_income'].apply(monthly_income_bin)
t2_monthly_income = working_df2['monthly_income_bin'].value_counts().sort_index()


monthly_income_df = pd.DataFrame({f'Below 300':t1_monthly_income, 
                       f'300-749':t2_monthly_income})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3708878486.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1['monthly_income_bin'] = working_df1['monthly_income'].apply(monthly_income_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3708878486.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2['monthly_income_bin'] = working_df2['monthly_income'].apply(monthly_income_bin)


In [27]:
#res_years_at_current_address
def res_years_at_current_address_bin(x):
    try:
        y = float(x)
        
        if y >= 10:
            return 'd [10, Inf)'
        elif y >= 5 and y < 10:
            return 'c [5,10)'
        elif y >= 1 and y < 5:
            return 'b [1,5)'
        elif y < 1:
            return 'a [-Inf,1)'        
    except:
        return x

working_df1.loc[:, 'res_years_at_current_address_bin'] = working_df1['res_years_at_current_address'].apply(res_years_at_current_address_bin)
t1_res_years_at_current_address = working_df1['res_years_at_current_address_bin'].value_counts().sort_index()

working_df2.loc[:, 'res_years_at_current_address_bin'] = working_df2['res_years_at_current_address'].apply(res_years_at_current_address_bin)
t2_res_years_at_current_address = working_df2['res_years_at_current_address_bin'].value_counts().sort_index()


res_years_at_current_address_df = pd.DataFrame({f'Below 300':t1_res_years_at_current_address, 
                       f'300-749':t2_res_years_at_current_address})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3452785598.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1.loc[:, 'res_years_at_current_address_bin'] = working_df1['res_years_at_current_address'].apply(res_years_at_current_address_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\3452785598.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2.loc[:, 'res_years_at_current_address_bin'] = working_df2['res_years_at_current_address'].apply(res_

In [28]:
#res_years_at_current_city
def res_years_at_current_city_bin(x):
    try:
        y = float(x)
        
        if y >= 10:
            return 'd [10, Inf)'
        elif y >= 5 and y < 10:
            return 'c [5,10)'
        elif y >= 1 and y < 5:
            return 'b [1,5)'
        elif y < 1:
            return 'a [-Inf,1)'        
    except:
        return x

working_df1.loc[:, 'res_years_at_current_city_bin'] = working_df1['res_years_at_current_city'].apply(res_years_at_current_city_bin)
t1_res_years_at_current_city = working_df1['res_years_at_current_city_bin'].value_counts().sort_index()

working_df2.loc[:, 'res_years_at_current_city_bin'] = working_df2['res_years_at_current_city'].apply(res_years_at_current_city_bin)
t2_res_years_at_current_city = working_df2['res_years_at_current_city_bin'].value_counts().sort_index()


res_years_at_current_city_df = pd.DataFrame({f'Below 300':t1_res_years_at_current_city, 
                       f'300-749':t2_res_years_at_current_city})

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\1064223899.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df1.loc[:, 'res_years_at_current_city_bin'] = working_df1['res_years_at_current_city'].apply(res_years_at_current_city_bin)
C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_17100\1064223899.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df2.loc[:, 'res_years_at_current_city_bin'] = working_df2['res_years_at_current_city'].apply(res_years_at_curren

In [40]:
#combine all results in one workbook

dist_categ_list = ['res_city', 'asset_category', 'asset_model', 'dealer', 'channel',
                  'res_state','industry','nature_of_business','res_resident_status',
                  'marital_status','manufacturer','occupation_type','product_description',
                  'branch_name', 'gender', 'asset_model_grp']

def dist_categ(variable):
    t1 = working_df1[variable].value_counts().sort_index()
    t2 = working_df2[variable].value_counts().sort_index()
    
    sheetname = f'{variable}'
    mix_df = pd.DataFrame({f'Good Customers count':t1,
                           f'Bad Customers count':t2})
    mix_df.to_excel(writer,sheet_name=sheetname, index = True)

def num_categ(variable):
    t1 = working_df1[variable].astype(float).value_counts().sort_index()
    t2 = working_df2[variable].astype(float).value_counts().sort_index()
    
    sheetname = f'{variable}'
    mix_df = pd.DataFrame({f'Good Customers count':t1,
                           f'Bad Customers count':t2})
    mix_df.to_excel(writer,sheet_name=sheetname, index = True)
    
with pd.ExcelWriter('PSI Temp_Jan_Refresh.xlsx', engine='openpyxl') as writer:
    for i in range (0,len(dist_categ_list)):
        dist_categ(dist_categ_list[i])
#     num_categ('period')
#     num_categ('converrate')
    num_categ('number_of_installments')
    loan_asset_cost_df.to_excel(writer,sheet_name='loan_asset_cost', index = True)
    down_pay_amt_df.to_excel(writer,sheet_name='down_pay_amt', index = True)
    num_categ('interest_rate')
    bureau_score_df.to_excel(writer,sheet_name='bureau_score', index = True)
    res_years_at_current_address_df.to_excel(writer,sheet_name='res_years_at_current_address', index = True)
    res_years_at_current_city_df.to_excel(writer,sheet_name='res_years_at_current_city', index = True)
    age_df.to_excel(writer,sheet_name='age', index = True)
    years_in_occup_df.to_excel(writer,sheet_name='years_in_occupation', index = True)
    total_income_df.to_excel(writer,sheet_name='total_income', index = True)
    monthly_income_df.to_excel(writer,sheet_name='monthly_income', index = True)
    dpratio_df.to_excel(writer,sheet_name='dp_ratio', index = True)
    #req_loan_amt_df.to_excel(writer,sheet_name='req_loan_amt', index = True)

In [41]:
print('DONE')

DONE


# Extra code for contingency

In [31]:
#combine all results in one workbook

# dist_categ_list = ['central_bank_check_verfication', 'other_checks','address_verification',
#                   'identification_verification','customer_risk_category','res_city_grp',
#                   'res_state_grp','industry_grp','nature_of_business','res_resident_status_grp',
#                   'marital_status','manufacturer_grp','occupation_type','product_description',
#                   'branch_grp']

# def dist_categ(variable):
#     t1 = working_df1[variable].value_counts().sort_index()
#     t2 = working_df2[variable].value_counts().sort_index()
#     t3 = working_df3[variable].value_counts().sort_index()
    
#     sheetname = f'{variable}'
#     mix_df = pd.DataFrame({f'CS B300 {variable} count':t1,
#                            f'CS 300-749 {variable} count':t2,
#                           f'CS above 750 {variable} count':t3})
#     mix_df.to_excel(writer,sheet_name=sheetname, index = True)

# def num_categ(variable):
#     t1 = working_df1[variable].astype(float).value_counts().sort_index()
#     t2 = working_df2[variable].astype(float).value_counts().sort_index()
#     t3 = working_df3[variable].astype(float).value_counts().sort_index()
    
#     sheetname = f'{variable}'
#     mix_df = pd.DataFrame({f'Below 300 {variable} count':t1,
#                            f'300-749 {variable} count':t2,
#                           f'above 750 {variable} count':t3})
#     mix_df.to_excel(writer,sheet_name=sheetname, index = True)
    
# with pd.ExcelWriter('PSI Temp.xlsx', engine='openpyxl') as writer:
#     for i in range (0,len(dist_categ_list)):
#         dist_categ(dist_categ_list[i])
# #     num_categ('period')
# #     num_categ('converrate')
#     df_deviation.to_excel(writer,sheet_name='deviation_count', index = True)
#     dpratio_df.to_excel(writer,sheet_name='dp_ratio', index = True)
#     disburse_amt_df.to_excel(writer,sheet_name='disburse_amt', index = True)
#     req_loan_amt_df.to_excel(writer,sheet_name='req_loan_amt', index = True)
#     final_sanc_amt_df.to_excel(writer,sheet_name='final_sanc_amt', index = True)
#     orig_emi_amt_df.to_excel(writer,sheet_name='orig_emi_amt', index = True)
#     down_pay_amt_df.to_excel(writer,sheet_name='down_pay_amt', index = True)
#     loan_asset_cost_df.to_excel(writer,sheet_name='loan_asset_cost', index = True)
#     df_verification_waived.to_excel(writer,sheet_name='verification_waived', index = True)
#     num_categ('number_of_installments')
#     age_df.to_excel(writer,sheet_name='age', index = True)
#     df_PerVeri_TelVer.to_excel(writer,sheet_name='PerVeri_TelVer', index = True)
#     years_in_occup_df.to_excel(writer,sheet_name='years_in_occupation', index = True)
#     total_income_df.to_excel(writer,sheet_name='total_income', index = True)
#     monthly_income_df.to_excel(writer,sheet_name='monthly_income', index = True)
#     tat_df.to_excel(writer,sheet_name='tat', index = True)
#     years_other_add_df.to_excel(writer,sheet_name='years_other_add', index = True)
#     df_ResiVeri_FieldVer.to_excel(writer,sheet_name='ResiVeri_FieldVer', index = True)
# #     mptir_df.to_excel(writer,sheet_name='mptir', index = True)
# #     mptir_df.to_excel(writer,sheet_name='mptir', index = True)